In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import bz2
import _pickle as cPickle


path = 'folder_with_raw_data';
# 0 - whole dataset, 1 - dataset for subsets
create_subset  = 1


df = pd.read_csv(path + 'kdd',header=None)

print("Total rows:", df.shape[0])
print("Total columns:", df.shape[1])

if create_subset==0:
    # one hot encoding for protocol -> +3 columns
    # print(df[1].unique())
    # print(len(df[1].unique()))
    one_hot_proto = pd.get_dummies(df[1])
    df = df.drop(1,axis = 1)
    df = df.join(one_hot_proto)

# one hot encoding for service -> +66 columns
# print(len(df[2].unique()))
one_hot_service = pd.get_dummies(df[2])
df = df.drop(2,axis = 1)
df = df.join(one_hot_service)
# print("Total columns:", df.shape[1])

# one hot encoding for flag -> +11 columns
# print(df[3].unique())
# print(len(df[3].unique()))
one_hot_flag = pd.get_dummies(df[3])
df = df.drop(3,axis = 1)
df = df.join(one_hot_flag)

#classes to remove

array = df[41].unique()
n = len(array)
classes_to_remove = []

for i in range(n):
    subset = df[df[41] == array[i]]
    rows = subset.shape[0]
    if rows<=800:
        classes_to_remove.append(array[i])

# print(classes_to_remove)
# remove classes
df = df[~df[41].isin(classes_to_remove)]

print("No. of rows without some classes:", df.shape[0])
print("Total number of features + target: ", df.shape[1])

# print(df)
# encoding for output - attack_cat
# print(df[41].unique())
le = LabelEncoder()
le.fit(df[41].unique())
df[41] = le.transform(df[41])

if create_subset==1:
    array = df[1].unique()
    array.sort()
    n = len(array)

    print("Number of rows for each protocol: (", n , "protocol )")

    for i in range(n):
        subset = df[df[1] == array[i]]
        rows = subset.shape[0]
        attack_rows = subset[subset[41] != le.transform(['normal.'])[0]].shape[0]
        print(array[i], " -> ", rows , ", attacks: ", attack_rows)

data = df

if create_subset == 0:
    # target
    target = np.array(data[41])

    # data
    data = data.drop([41],axis= 1)
    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)

    print("Final features: ", data.shape[1])

    f = bz2.BZ2File("kdd.pbz2", "wb")
    cPickle.dump((data,target),f)
    f.close()

if create_subset == 1:
    # target
    target = np.array(data[41])

    # subsets
    subsets = np.array(data[1])

    # data
    data = data.drop([41],axis= 1)
    data = data.drop([1],axis= 1)
    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)

    print("Final features: ", data.shape[1])

    f = bz2.BZ2File("kdd_subsets.pbz2", "wb")
    cPickle.dump((data,target,subsets),f)
    f.close()

Total rows: 494021
Total columns: 42
No. of rows without some classes: 493347
Total number of features + target:  117
Number of rows for each protocol: ( 3 protocol )
icmp  ->  283235 , attacks:  281947
tcp  ->  189786 , attacks:  112973
udp  ->  20326 , attacks:  1149
Final features:  115
